## Repliikide lausestamine test5

In [36]:
from estnltk import Text
import sqlite3
from estnltk.converters import json_to_text
from estnltk.converters import text_to_json
from estnltk_core.layer_operations import split_by_sentences

Tühi järjend repliikide hoiustamiseks:

In [37]:
corpus = []

Tühi järjend repliikide ID-de jaoks:

In [38]:
line_ids = []

Andmebaasi tabelist loetakse sisse repliikidele vastavad Text-objektid:

In [39]:
con = sqlite3.connect("media_data.db")

In [40]:
cur = con.cursor()

In [41]:
for row in cur.execute("SELECT ID, line FROM lines"):
    line_id = row[0]
    line = json_to_text(json_text=row[1])
    corpus.append(line)
    line_ids.append(line_id)

In [42]:
con.close()

In [43]:
len(corpus)

391

In [44]:
test_text = corpus[14]
display(test_text)

Text(text='Kes peab end jätkab Tartus Rein kilk Tallinnas Hans H luik ja priit olemegi ju meie rubriik uued nimed ja meie selle nädala uus nimi on Vadim Belov professor kes on siis Tallinna Tallinnasse uusi kandidaate aselinnapeaks sotside poolt teatavasti viibis sealt.\nTa läks ise tagasi astus ise tagasi siiamaani ta jätkab oma tegevust seal Vadim peab roht seal on siis endine telekanali PBK saatejuht endine venekeelse Delfi toimetaja endine ühe venekeelse ajalehe.\nTöötaja et inimene kes räägib kes pärineb intelligentsus perekonnast kelle isa on juba olnud tegutseb.\nTõlkena oli vist ka Tallinna ülikooliga.\nKes on ise tegutsenud ajakirjaniku vana ajal mil pronksiöökirjad olid.\nTaevasse tõusnud ja võttis ka üsna üheselt seisukohti pronksiosas.')

In [45]:
test_text.ner

Layer(name='ner', attributes=('nertag',), spans=SL[EnvelopingSpan(['Tartus'], [{'nertag': 'LOC'}]),
EnvelopingSpan(['Tallinnas'], [{'nertag': 'LOC'}]),
EnvelopingSpan(['Hans'], [{'nertag': 'PER'}]),
EnvelopingSpan(['H'], [{'nertag': 'ORG'}]),
EnvelopingSpan(['Vadim', 'Belov'], [{'nertag': 'ORG'}]),
EnvelopingSpan(['Tallinna'], [{'nertag': 'LOC'}]),
EnvelopingSpan(['Tallinnasse'], [{'nertag': 'LOC'}]),
EnvelopingSpan(['Vadim'], [{'nertag': 'PER'}]),
EnvelopingSpan(['PBK'], [{'nertag': 'ORG'}]),
EnvelopingSpan(['Delfi'], [{'nertag': 'ORG'}]),
EnvelopingSpan(['Tallinna', 'ülikooliga'], [{'nertag': 'ORG'}])])

Kuna repliigid on suhteliselt kirjakeelsed (laused algavad suure algustähega ning lõppevad lauselõpumärgiga), lausestan esialgu split_by_sentences abil:

In [46]:
line_sentences = split_by_sentences(text=test_text,
                           layers_to_keep=list(test_text.layers),
                           trim_overlapping=True
                           )

In [47]:
for sentence in line_sentences:
    if len(sentence.ner) > 0:
        display(sentence)
        display(sentence.ner)

Text(text='Kes peab end jätkab Tartus Rein kilk Tallinnas Hans H luik ja priit olemegi ju meie rubriik uued nimed ja meie selle nädala uus nimi on Vadim Belov professor kes on siis Tallinna Tallinnasse uusi kandidaate aselinnapeaks sotside poolt teatavasti viibis sealt.')

Layer(name='ner', attributes=('nertag',), spans=SL[EnvelopingSpan(['Tartus'], [{'nertag': 'LOC'}]),
EnvelopingSpan(['Tallinnas'], [{'nertag': 'LOC'}]),
EnvelopingSpan(['Hans'], [{'nertag': 'PER'}]),
EnvelopingSpan(['H'], [{'nertag': 'ORG'}]),
EnvelopingSpan(['Vadim', 'Belov'], [{'nertag': 'ORG'}]),
EnvelopingSpan(['Tallinna'], [{'nertag': 'LOC'}]),
EnvelopingSpan(['Tallinnasse'], [{'nertag': 'LOC'}])])

Text(text='Ta läks ise tagasi astus ise tagasi siiamaani ta jätkab oma tegevust seal Vadim peab roht seal on siis endine telekanali PBK saatejuht endine venekeelse Delfi toimetaja endine ühe venekeelse ajalehe.')

Layer(name='ner', attributes=('nertag',), spans=SL[EnvelopingSpan(['Vadim'], [{'nertag': 'PER'}]),
EnvelopingSpan(['PBK'], [{'nertag': 'ORG'}]),
EnvelopingSpan(['Delfi'], [{'nertag': 'ORG'}])])

Text(text='Tõlkena oli vist ka Tallinna ülikooliga.')

Layer(name='ner', attributes=('nertag',), spans=SL[EnvelopingSpan(['Tallinna', 'ülikooliga'], [{'nertag': 'ORG'}])])

In [48]:
con = sqlite3.connect("media_data.db")
cur = con.cursor()

Andmebaasi lisatakse uus tabel lausete jaoks:

In [50]:
cur.execute("CREATE TABLE sentences(ID INTEGER PRIMARY KEY, line_ID INTEGER, sentence TEXT, named_entity INTEGER)")

Loodud tabelisse lisatakse kõik laused, eraldi märge lisatakse veergu named_entity, kas lause sialdab mõnda nimeüksust või mitte:

In [51]:
for i in range(len(corpus)):
    line_sentences = split_by_sentences(text=corpus[i], 
                           layers_to_keep=list(corpus[i].layers),
                           trim_overlapping=True
                           )
    for sentence in line_sentences:
        sentence.meta['line_number'] = corpus[i].meta['line_number']
        sentence.meta['person'] = corpus[i].meta['person']
        sentence.meta['date'] = corpus[i].meta['date']
        sentence_json = text_to_json(sentence)
        if len(sentence.ner) > 0:
            cur.execute("""INSERT INTO sentences
                                    (line_ID, sentence, named_entity)
                                    VALUES (?, ?, ?);""", (line_ids[i], sentence_json, 1))
        else:
            cur.execute("""INSERT INTO sentences
                                    (line_ID, sentence, named_entity)
                                    VALUES (?, ?, ?);""", (line_ids[i], sentence_json, 0))
        con.commit()

In [52]:
con.close()